# ASSP + Diameter 

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.3.1`
import $ivy.`sh.almond::almond-spark:0.6.0`

import org.apache.spark.sql._

import org.apache.log4j.{Level, Logger}
import org.apache.spark.rdd.RDD
import org.apache.spark.{SparkConf, SparkContext}

import $ivy.$                                  

import $ivy.$                              


import org.apache.spark.sql._


import org.apache.log4j.{Level, Logger}

import org.apache.spark.rdd.RDD

import org.apache.spark.{SparkConf, SparkContext}

In [2]:
Logger.getLogger("org").setLevel(Level.OFF)

In [3]:
val spark = {
  NotebookSparkSession
    .builder()
    .master("local[*]")
    .getOrCreate()
}

Loading spark-stubs
Getting spark JARs
Creating SparkSession


Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


Spark UI

spark: SparkSession = org.apache.spark.sql.SparkSession@4f2c7b64

In [4]:
def sc = spark
    .sparkContext

defined function sc

In [5]:
def generateGraphRDD(context: SparkContext, inputFile: String, separator: String): RDD[(String, Iterable[String])] = {
    val graph = context.textFile(inputFile)
        .map { line => val tokens = line.split(separator)
            (tokens(0), tokens(1))
        }
        .groupByKey()
        .cache()

    graph
}

defined function generateGraphRDD

In [6]:
val GraphRDD = generateGraphRDD(sc, "../input/test_input.txt", " ")

GraphRDD: RDD[(String, Iterable[String])] = ShuffledRDD[3] at groupByKey at cmd4.sc:6

In [7]:
GraphRDD.foreach(println)

map at cmd4.sc:3

2 / 2

foreach at cmd6.sc:1

2 / 2

(4,CompactBuffer(5))
(6,CompactBuffer(4, 7))
(0,CompactBuffer(1, 2))
(2,CompactBuffer(3))
(5,CompactBuffer(6))
(1,CompactBuffer(2, 3))


In [8]:
def getK(GraphRDD: RDD[(String, Iterable[String])]): Int = {
    val k = GraphRDD
        .count()
        .toInt

    k
}

defined function getK

In [9]:
val k = getK(GraphRDD)

count at cmd7.sc:3

2 / 2

k: Int = 6

In [10]:
println(k)

6


In [11]:
val edgeWeight = 1

edgeWeight: Int = 1

In [12]:
var DistancesRDD = GraphRDD
    .flatMap {
        case (fromId, adjList) => adjList
            .map(adjId => (adjId, (fromId, edgeWeight)))
}

DistancesRDD: RDD[(String, (String, Int))] = MapPartitionsRDD[4] at flatMap at cmd11.sc:2

In [13]:
DistancesRDD.foreach(println)

foreach at cmd12.sc:1

2 / 2

(5,(4,1))
(4,(6,1))
(7,(6,1))
(1,(0,1))
(2,(0,1))
(6,(5,1))
(3,(2,1))
(2,(1,1))
(3,(1,1))


In [14]:
def updateDistances(tuple: (String, (Option[Iterable[String]], (String, Int)))):
Iterable[((String, String), Int)] = {
    tuple match {
        case (toId, (Some(adjList), (fromId, distance))) => adjList
                .map(newId => ((newId, fromId), edgeWeight + distance)) ++ List(((toId, fromId), distance))
                    case (toId, (None, (fromId, distance))) => List(((toId, fromId), distance))
    }
}
.filter {
    case ((toId, fromId), _) => !toId.equals(fromId) 
} // Don't keep circular distances

defined function updateDistances

In [15]:
for (iteration <- 0 to k) {
    DistancesRDD = GraphRDD
        .rightOuterJoin(DistancesRDD) // (toId, (Option[adjList], (fromId, distance)))
        .flatMap(x => updateDistances(x))
        .reduceByKey((x, y) => java.lang.Math.min(x, y)) // Only keep min distance for any (to, from) pair
        .map { 
            case ((toId, fromId), distance) => (toId, (fromId, distance))
        }
}

In [16]:
DistancesRDD.foreach(println)

flatMap at cmd11.sc:2

2 / 2

flatMap at cmd14.sc:4

2 / 2

map at cmd14.sc:6

...

flatMap at cmd14.sc:4

2 / 2

foreach at cmd15.sc:1

2 / 2

(2,(1,1))
(5,(6,2))
(7,(5,2))
(7,(4,3))
(2,(0,1))
(4,(5,2))
(6,(4,2))
(3,(1,1))
(7,(6,1))
(1,(0,1))
(3,(2,1))
(6,(5,1))
(4,(6,1))
(5,(4,1))
(3,(0,2))


In [17]:
def asspRDD(GraphRDD: RDD[(String, Iterable[String])]) = {
    val k = getK(GraphRDD)

    // Distances structure: (toId, (fromId, distance))
    // This data will change each iteration
    // Set all distances for "first hop" to 1
    var DistancesRDD = GraphRDD.flatMap { case (fromId, adjList) =>
      adjList.map(adjId => (adjId, (fromId, edgeWeight)))
    }

    // How to calculate the shortest path?
    // All origin userIDs are nodes in the graph.  If there exists a shortest path to anywhere, it must start at an ID in the graph
    // On a shortest path, for all nodes in the path, the path is also the shortest path for those nodes
    // So we really just need to build up the paths from every possible starting ID?
    // And then after convergence go through and output the final results?  How?
    // Will every node get covered?
    for (iteration <- 0 to k) {
      DistancesRDD = GraphRDD.rightOuterJoin(DistancesRDD) // (toId, (Option[adjList], (fromId, distance)))
        .flatMap(x => updateDistances(x))
        .reduceByKey((x, y) => java.lang.Math.min(x, y)) // Only keep min distance for any (to, from) pair
        .map { case ((toId, fromId), distance) => (toId, (fromId, distance)) }
    }

    DistancesRDD
  }

defined function asspRDD

In [18]:
val DistancesRDD = asspRDD(GraphRDD)

count at cmd7.sc:3

2 / 2

DistancesRDD: RDD[(String, (String, Int))] = MapPartitionsRDD[89] at map at cmd16.sc:21

In [19]:
DistancesRDD.foreach(println)

flatMap at cmd16.sc:7

2 / 2

flatMap at cmd16.sc:19

2 / 2

map at cmd16.sc:21

...

flatMap at cmd16.sc:19

2 / 2

foreach at cmd18.sc:1

2 / 2

(2,(1,1))
(5,(6,2))
(7,(5,2))
(7,(4,3))
(6,(4,2))
(7,(6,1))
(1,(0,1))
(3,(2,1))
(6,(5,1))
(4,(6,1))
(5,(4,1))
(2,(0,1))
(4,(5,2))
(3,(1,1))
(3,(0,2))


In [20]:
val diameter = DistancesRDD
      .sortBy(_._2._2, ascending = false)
      .take(1)

sortBy at cmd19.sc:2

2 / 2

sortBy at cmd19.sc:2

2 / 2

take at cmd19.sc:3

1 / 1

diameter: Array[(String, (String, Int))] = Array(("7", ("4", 3)))

In [21]:
diameter.foreach(println)

(7,(4,3))


In [22]:
def getDiameter(context: SparkContext, GraphRDD: RDD[(String, Iterable[String])]) = {

    val DistancesRDD = asspRDD(GraphRDD)

//    val diameter = graph
//      .collect
//      .maxBy(_._2._2)
//      .collect.maxBy(_._2)

    val diameter = DistancesRDD
      .sortBy(_._2._2, ascending = false)
      .take(1)

    context.parallelize(diameter)
  }

defined function getDiameter

In [23]:
val diameter = getDiameter(sc, GraphRDD)

count at cmd7.sc:3

2 / 2

flatMap at cmd16.sc:7

2 / 2

flatMap at cmd16.sc:19

...

sortBy at cmd21.sc:11

2 / 2

take at cmd21.sc:12

1 / 1

diameter: RDD[(String, (String, Int))] = ParallelCollectionRDD[143] at parallelize at cmd21.sc:14

In [24]:
diameter.foreach(println)

foreach at cmd23.sc:1

6 / 6

(7,(4,3))
